In [1]:
import pandas as pd
import numpy as np
from astropy.io import fits

import matplotlib.pyplot as plt
from astropy.visualization import make_lupton_rgb
plt.style.use('dark_background')

In [16]:
#Response to Phil - Specific ~0

filepath = '/Users/jimenagonzalez/research/DSPL/Citizen_science_project/Intersection/'
data_inter = pd.read_csv(filepath + 'intersection.csv')
data_inter = data_inter.dropna()  # Remove rows with NaN values
data_inter = data_inter.replace([np.inf, -np.inf], np.nan).dropna()

#80% of len(data_inter) is 602578

data_inter.sort_values(by='K_SCORE', inplace=True)
data_tmp = data_inter.head(602578)
print('Highest Rojas: ', data_tmp['K_SCORE'].iloc[-1])

data_inter.sort_values(by='score_sims', inplace=True)
data_tmp = data_inter.head(602578)
print('Highest Jacobs: ', data_tmp['score_sims'].iloc[-1])

data_inter.sort_values(by='SINGLE', inplace=True)
data_tmp = data_inter.head(602578)
print('Highest Gonzalez: ', data_tmp['SINGLE'].iloc[-1])

#maximum value of prob: 0.007

Highest Rojas:  0.006727251
Highest Jacobs:  0.0033
Highest Gonzalez:  0.001


In [2]:
data_zoo = pd.read_csv('data/complete_data.csv')

filepath = '/Users/jimenagonzalez/research/DSPL/Citizen_science_project/Intersection/data/in_sled2.fits'
hdu_list = fits.open(filepath)
images = hdu_list[1].data
data = pd.DataFrame(hdu_list[2].data)
hdu_list.close()

data.replace('None', np.nan, inplace=True)
data = data.astype({'COADD_OBJECT_ID': int, 'K_RA': float, 'K_DEC': float, 'K_SCORE': float, 'Y3_COADD_OBJECT_ID': int,
       'J_RA': float, 'J_DEC': float, 'score_sims': float, 'score_real': float, 'score_both': float, 'G_RA': float,
       'G_DEC': float, 'SINGLE': float, 'RING': float, 'SMOOTH': float, 'COMPANIONS': float, 'SDSS_SPIRALS': float,
       'DES_SPIRALS': float, 'CROWDED': float, 'ARTIFACTS': float, 'MOST_NEGATIVES': float, 'index': int, 'ra': float,
       'dec': float, 'name': str, 'score': float, 'image_sep': float, 'info': str, 'n_img': float, 'flag': str,
       'image_conf': str, 'lens_type': str, 'source_type': str, 'contaminant_type': str, 'papers': str})

#Getting the images
data.set_index('COADD_OBJECT_ID', inplace=True)
data_zoo.set_index('COADD_OBJECT_ID', inplace=True)
# Reindex 'data' to match the index of 'data_zoo'
data = data.reindex(data_zoo.index)
# Reset index if needed
data.reset_index(inplace=True)
data_zoo.reset_index(inplace=True)
images = images[data.index]

In [3]:
# For most important paragraph. Most high-confidence candidates are recovered by the three works combined.

print(len(data_zoo))

# Candidates with ambiguous confidence
print(' ')
data_tmp = data_zoo[(data_zoo['average'] >= 1) & (data_zoo['average'] < 1.5)]
print(len(data_tmp))
print(len(data_tmp[(data_tmp['score_sims'] < 0.8) & (data_tmp['K_SCORE'] < 0.8) & (data_tmp['SINGLE'] < 0.8)]))
print('Percentage moderate candidates with all ML scores < 0.8: {:.2f}%'.format(88/275*100))

# Candidates with moderate confidence
print(' ')
data_tmp = data_zoo[(data_zoo['average'] >= 1.5) & (data_zoo['average'] < 2)]
print(len(data_tmp))
print(len(data_tmp[(data_tmp['score_sims'] < 0.8) & (data_tmp['K_SCORE'] < 0.8) & (data_tmp['SINGLE'] < 0.8)]))
print('Percentage moderate candidates with all ML scores < 0.8: {:.2f}%'.format(16/93*100))

# Candidates with high confidence
print(' ')
data_tmp = data_zoo[(data_zoo['average'] >= 2)]
print(len(data_tmp))
print(len(data_tmp[(data_tmp['score_sims'] < 0.8) & (data_tmp['K_SCORE'] < 0.8) & (data_tmp['SINGLE'] < 0.8)]))
print('Percentage moderate candidates with all ML scores < 0.8: {:.2f}%'.format(1/48*100))

1651
 
275
88
Percentage moderate candidates with all ML scores < 0.8: 32.00%
 
93
16
Percentage moderate candidates with all ML scores < 0.8: 17.20%
 
48
1
Percentage moderate candidates with all ML scores < 0.8: 2.08%


In [4]:
# Studying the recovery individually: successive works recover more and more candidates

print(len(data_zoo))

# Candidates with moderate-high confidence
data_tmp = data_zoo[data_zoo['average'] >= 1.75]
print(len(data_tmp))

print(' ')
print(len(data_tmp[data_tmp['score_sims'] < 0.8]))
print('Percentage Jacobs ml score < 0.8: {:.2f}%'.format(36/74*100))

# Candidates with moderate confidence
print(' ')
print(len(data_tmp[data_tmp['K_SCORE'] < 0.8]))
print('Percentage Rojas ml score < 0.8: {:.2f}%'.format(23/74*100))

# Candidates with high confidence
print(' ')
print(len(data_tmp[data_tmp['SINGLE'] < 0.8]))
print('Percentage Gonzalez ml score < 0.8: {:.2f}%'.format(9/74*100))

1651
74
 
36
Percentage Jacobs ml score < 0.8: 48.65%
 
23
Percentage Rojas ml score < 0.8: 31.08%
 
9
Percentage Gonzalez ml score < 0.8: 12.16%


In [5]:
# Candidates with moderate-high confidence, for conclusion
print(' ')
data_tmp = data_zoo[data_zoo['average'] >= 1.75]
print(len(data_tmp))
print(len(data_tmp[(data_tmp['score_sims'] < 0.8) & (data_tmp['K_SCORE'] < 0.8) & (data_tmp['SINGLE'] < 0.8)]))
print('Percentage moderate-high candidates with all ML scores < 0.8: {:.2f}%'.format(5/74*100))

print(' ')
data_tmp = data_zoo[data_zoo['average'] >= 1.75]
print(len(data_tmp))
print(len(data_tmp[(data_tmp['score_sims'] >= 0.8) | (data_tmp['K_SCORE'] >= 0.8) | (data_tmp['SINGLE'] >= 0.8)]))
print('Percentage moderate-high candidates with all ML scores < 0.8: {:.2f}%'.format(5/74*100))

 
74
5
Percentage moderate-high candidates with all ML scores < 0.8: 6.76%
 
74
69
Percentage moderate-high candidates with all ML scores < 0.8: 6.76%
